# Importing libraries

In [ ]:
import os, glob, csv, subprocess, sys, re, operator
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd

# Defining repository and directories

In [ ]:
project = ""      # put the cloned repository name in the project
userhome = os.path.expanduser('~')
repository = userhome + r'/DifferentDiffAlgorithms/SZZ/datasource/' + project + '/'
analyze_dir = userhome + r'/DifferentDiffAlgorithms/SZZ/projects_analyses/' + project + '/'

# Defining function to execute git command

In [ ]:
def execute_command(cmd, work_dir):
   #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Loading the datasets

Specifying column names

In [ ]:
colnames = ['bug_id','bugfix_commitID','parent_id','filepath','filename_myers','filename_histogram','#deletions_myers','#deletions_histogram']

In [ ]:
diffbugs = pd.read_csv(analyze_dir + '02_diff_extraction/03_file_having_deletedlines/buggyfiles_from_both_algorithms.csv')
diffbugs

# Applying "git blame" for files

Define pattern

In [ ]:
pattern1 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+(?P<filename>[^\s]+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>.*)')
pattern2 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>.*)')

Define header of table

In [ ]:
header1 = ['bugintro_commitid', 'filename', 'committer', 'date', 'time', 'line_number', 'code']
header2 = ['bugintro_commitid', 'committer', 'date', 'time', 'line_number', 'code']

In [ ]:
#Define directory
blame_dirs = analyze_dir + '03_annotate/01_annotated_files/'

# Annotating files

In [ ]:
errorfile_hist = []
errorfile_myers = []
df_blame = []
for cc in range(0, len(diffbugs)):
    sys.stdout.write("\rApplying git blame: %i" % (cc+1) + " / %i" % (len(diffbugs)))
    sys.stdout.flush()
    
    data = diffbugs.iloc[cc]
    
    parent = data[2]
    filepath = data[3]
    
    bugid = data[0]
    fixcid = data[1]
    
    fn = ((data[3].split('/'))[-1:])[0]
    fname = fn + "_" + data[1] + "-" + data[2][:10] + "_" + data[0] 
    
    blametxt = analyze_dir + '03_annotate/01_annotated_files/listof_diff_n_annotated_files/blame_' + fname + '_' + str(cc+1) + '.txt'
    
    #checkout parentID and applying git blame
    checkout = "git checkout -f " + parent
    blame = "git blame " + filepath + " > " + blametxt
    
    execute_command(checkout, repository)
    execute_command(blame, repository)
    
    b = open(blametxt, encoding="utf8", errors='ignore')
    c = b.read().split('\n')

    #annotating myers produced files
    m_bugnum = data[6]    
    if m_bugnum != 0:
        mmm = 'blamemyers_' + fname + '_' + str(cc+1) + '.csv'
        name = analyze_dir + '03_annotate/01_annotated_files/blame_myers/csv/' + mmm
        with open(name, 'w', newline='') as f:
            writer = csv.writer(f)
            try:
                try:
                    pattern1.match(c[0]).groups()
                    writer.writerow(header1)
                    for line in c:
                        writer.writerow(pattern1.match(line).groups())
                except:
                    pattern2.match(c[0]).groups()
                    writer.writerow(header2)
                    for line in c:
                        writer.writerow(pattern2.match(line).groups())
            except:
                if os.stat(blametxt).st_size == 0:
                    writer.writerow(header1)
                    tmp = [bugid, fixcid, parent, fn, filepath, fname + '_' + str(cc+1), m_bugnum]
                    errorfile_myers.append(tmp)
                    print (' : ' + fname + '_' + str(cc+1) + ' --> file not found')
                pass
    else:
        mmm = "-"
            
    #annotating histogram produced files
    h_bugnum = data[7]
    if h_bugnum != 0:
        hhh = 'blamehistogram_' + fname + '_' + str(cc+1) + '.csv'
        name = analyze_dir + '03_annotate/01_annotated_files/blame_histogram/csv/' + hhh
        with open(name, 'w', newline='') as f:
            writer = csv.writer(f)
            try:
                try:
                    pattern1.match(c[0]).groups()
                    writer.writerow(header1)
                    for line in c:
                        writer.writerow(pattern1.match(line).groups())
                except:
                    pattern2.match(c[0]).groups()
                    writer.writerow(header2)
                    for line in c:
                        writer.writerow(pattern2.match(line).groups())
            except:
                if os.stat(blametxt).st_size == 0:
                    writer.writerow(header1)
                    tmp = [bugid, fixcid, parent, fn, filepath, fname + '_' + str(cc+1), h_bugnum]
                    errorfile_hist.append(tmp)
                    print (' : ' + fname + '_' + str(cc+1) + ' --> file not found')
                pass
    else:
        hhh = "-"

    tempo = [data[0],data[1],data[2],data[3],data[4],data[5],mmm,hhh,data[6],data[7]]
    df_blame.append(tempo)
    
#exit from checkout
excheck = "git checkout -f origin"
execute_command(excheck, repository)

c = 1
for file in glob.iglob(analyze_dir + '03_annotate/01_annotated_files/listof_diff_n_annotated_files/*', recursive=True):
    c += 1
    try:
        os.remove(file)
    except FileNotFoundError:
        print ("\nError: %s" % (file))
        
print ("\n'git blame' implementation is complete")

In [ ]:
#Save data_frame file into CSV file
with open(analyze_dir + '03_annotate/01_annotated_files/listof_diff_n_annotated_files/diff_n_blame_combination_files.csv',
          'w') as dfcsv:
    header = ['bug_id','bugfix_commitID','parent_id','filepath','diff_myers_file','diff_histogram_file',
              'blame_myers_file','blame_histogram_file','#deletions_myers','#deletions_histogram']
    writers = csv.writer(dfcsv)
    writers.writerow(header)
    for file in df_blame:
        writers.writerow(file)
print ("The csv file has been created")